# VinaScreen
For screening multiple ligands
Requirements: 
Ligands: sdf, smi or pdb files
Receptor: PDB files of receptors in receptor folder
Parameters: config.txt of files specifying binding site, exhaustiveness and number of poses per ligand


## Convert ligands to pdbqt
!! output will be in ligand folder. make sure it is empty or previous run renamed

In [1]:
!python pdbqt_converter.py /home/nikka/Frag_output/top10_Ligs/top10Ligs.sdf

Detecting file format: sdf
Reading molecules from: /home/nikka/Frag_output/top10_Ligs/top10Ligs.sdf
Found 41 molecules
Output directory: ligand
Processed: 41/41 (converted: 41, skipped: 0) Rate: 72.7 mol/s ETA: 0s
CONVERSION SUMMARY
Total molecules processed: 41
Successfully converted: 41
Skipped (errors/timeouts): 0
Success rate: 100.0%
Total time: 0.6 seconds
Output files saved in: ligand/


## Run docking screen
do not forget to edit config.txt!!!!!
rename output

In [12]:
!python run_vinascreen.py

VinaScreen High-Throughput Docking Pipeline (main.py)

Step 1: Validating environment setup...
Validating VinaScreen environment...
✓ Found Vina executable: vina_1.2.7_linux_x86_64
✓ Found ligand folder with 41 ligand files
✓ Found receptor folder with 1 receptor files
✓ Found and validated config.txt file
✓ Created output folder: output
✓ Environment validation successful!

Step 2: Loading inputs...
  41 ligands loaded
  1 receptors loaded

Step 3: Initializing CSV reporter and progress tracker...
  Output CSV: vinascreen_results.csv
  Total docking jobs: 41

Step 4: Launching docking runs...

Processed: 41/41 (kept: 41, skipped: 0)

Step 5: Summary
Docking complete. 41 jobs processed.
Results saved to: vinascreen_results.csv


# Convert VinaScreen pdbqt output to smiles

In [2]:
import os
import re
import pandas as pd
from openbabel import pybel

def parse_vina_pdbqt(filename):
    """Extract models and scores from vina PDBQT output file."""
    results = []
    with open(filename, 'r') as f:
        content = f.read()
    # Split models: each model starts with "MODEL" and ends with "ENDMDL"
    models = re.split('(MODEL.*?ENDMDL)', content, flags=re.DOTALL)
    model_number = 1
    for model in models:
        if 'MODEL' in model and 'ENDMDL' in model:
            # Extract vina score (affinity) from REMARK line
            score_match = re.search(r'REMARK VINA RESULT:\s*([-.\d]+)', model)
            if score_match:
                score = float(score_match.group(1))
            else:
                score = None
            # Write each pose to temp pdbqt for conversion
            temp_name = '_temp_model.pdbqt'
            with open(temp_name, 'w') as out:
                out.write(model)
            # Use openbabel to convert to SMILES
            mols = list(pybel.readfile("pdbqt", temp_name))
            if mols:
                smiles = mols[0].write(format="smi").split()[0]
            else:
                smiles = None
            os.remove(temp_name)
            results.append((model_number, score, smiles))
            model_number += 1
    return results

data = []
folder = "output_PDL1G1" #path to folder
for fname in os.listdir(folder):
    if fname.endswith(".pdbqt"):
        full_path = os.path.join(folder, fname)
        basename = os.path.splitext(fname)[0]
        models = parse_vina_pdbqt(full_path)
        for model_id, vina_score, smiles in models:
            row_id = f"{basename}_{model_id}"
            data.append({'ID': row_id, 'SMILES': smiles, 'vina_score': vina_score})

# Output as CSV
df = pd.DataFrame(data)

# Sort to ensure lowest model number comes first for each SMILES
df['model_num'] = df['ID'].str.extract(r'_(\d+)$').astype(int)
df.sort_values(by=['SMILES', 'model_num'], inplace=True)
# Drop duplicates keeping lowest model number (first occurrence)
df_no_dupes = df.drop_duplicates(subset=["SMILES"], keep='first')
# Drop auxiliary column before saving
df_no_dupes = df_no_dupes.drop(columns=['model_num'])
df_no_dupes.to_csv("vina_PDL1G1.csv", index=False) #change file name


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is pdL1G1_PdL1G1_0250)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is pdL1G1_PdL1G1_0250)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is pdL1G1_PdL1G1_0250)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is pdL1G1_PdL1G1_0320)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is pdL1G1_PdL1G1_0320)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is pdL1G1_PdL1G1_0320)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is pdL1G1_PdL1G1_0202)

*** Op